In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os, time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Early Stopping

- <span style = 'font-size:1.3em;line-height:1.5em'>Early Stopping은 다음과 같은 방식으로 진행됩니다.</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>(1) 매 epoch마다 train을 진행합니다.</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>(2) 한 epoch에 대해 train이 끝나면 validation set에서 현재까지 학습된 모델로 loss를 계산합니다.</span>
        - <span style = 'font-size:1.1em;line-height:1.5em'>val_loss = loss_func(y_val, y_val_est)</span>

    - <span style = 'font-size:1.2em;line-height:1.5em'>(3) 현재의 validation loss가 이제까지의 validation loss의 최소값보다 연속으로 n번 크게 되면 학습을 멈춘다</span>
        - <span style = 'font-size:1.1em;line-height:1.5em'>val_loss > best_val_loss (n consecutive times) --> stop training</span>

## 실험을 CPU에서? GPU에서?

In [2]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)

cpu


## 1. 모델을 클래스 형식으로 선언합니다.

In [3]:
class MyNet(nn.Module):
    def __init__(self, dim_in=784, dim_h1=50, dim_h2=100, dim_out=10):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(dim_in,dim_h1)
        self.fc2 = nn.Linear(dim_h1,dim_h2)
        self.fc3 = nn.Linear(dim_h2,dim_out)
        self.apply(self._init_weights) # 모델을 만들때, self._init_weights()를 호출하여 parameter 초기화
        
    def _init_weights(self, submodule):
        if isinstance(submodule, nn.Linear): # submodule이 nn.Linear에서 생성된 객체(혹은 인스턴스이면)
            nn.init.kaiming_normal_(submodule.weight) #해당 submodule의 weight는 He Initialization으로 초기화
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01) # 해당 submodule의 bias는 0.01로 초기화
            
        
    def forward(self, x):
        h1 = self.fc1(x)
        h1 = F.relu(h1)
        
        h2 = self.fc2(h1)
        h2 = F.relu(h2)
        
        out = self.fc3(h2)
        # F.cross_entropy = F.log_softmax + F.nll_loss
        # 뒤에서 cross_entropy를 사용하려면, 여기서 softmax 빼야됩니다.
        
        return out

## 2. train() 함수

In [4]:
def train(model, data_loader, optimizer, criterion, device):
    model.train() # 모델을 학습모드로! Dropout이 있는 모델을 학습할 때, 반드시 필요함
    trn_loss = 0
    for i, (x, y) in enumerate(data_loader):
        # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
        x = x.view(-1, 784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
        y = y.to(device)
        
        # Step 2. gradient 초기화
        optimizer.zero_grad()
        
        # Step 3. Forward Propagation
        y_pred_prob = model(x)
        
        # Step 4. Loss Calculation
        loss = criterion(y_pred_prob, y)
        
        # Step 5. Gradient Calculation (Backpropagation)
        loss.backward()
        
        # Step 6. Update Parameter (by Gradient Descent)
        optimizer.step()
        
        # Step 7. trn_loss 변수에 mini-batch loss를 누적해서 합산
        trn_loss += loss.item()
        
    # Step 8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

## 3. evaluate() 함수

In [5]:
def evaluate(model, data_loader, optimizer, criterion, device):
    model.eval() # 모델을 평가모드로! Dropout이 있는 모델을 학습할 때, 반드시 필요함
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.view(-1,784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = criterion(y_pred_prob, y)
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

## 4. 매 Epoch에 드는 시간 측정

In [6]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 5. 학습하기

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [7]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
trn_dset = datasets.MNIST(root=data_path, train=True, transform=transform, download=True)
tst_dset = datasets.MNIST(root=data_path, train=False, transform=transform, download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [9]:
# without batchnorm
model = MyNet(dim_in=784, dim_h1=50, dim_h2=100, dim_out=10)
model = model.to(device)

- <span style = 'font-size:1.2em;line-height:1.5em'>loss function 정의하기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>예전 실습 파일에는 손실 함수인 F.nll_loss()가 train(), evaluate() 함수 안에서 바로 사용되었음</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>이번 손실 함수는 train(), evaluate() 함수 밖에서 정의하고 안에서는 정의한 함수를 사용하는 방식으로</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>어떻게 사용해도 상관없으니, 편한대로 사용하세요.</span>

In [10]:
loss_func = nn.CrossEntropyLoss(reduction='sum')

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [11]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [12]:
N_EPOCHS = 50
LR = 0.01
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [13]:
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기 <b>(weight decay부분이 여기 들어갑니다.)</b></span>

In [14]:
my_opt = optim.Adam(model.parameters(), lr=LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [15]:
best_val_loss = float('inf')
n_patience = 3
n_violence = 0

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     criterion=loss_func,
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=tst_loader, 
                                  optimizer=my_opt, 
                                  criterion=loss_func,
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrn Loss: {trn_loss:.3f} | Val Loss: {val_loss:.3f} | Val Acc: {100*accuracy:.3f}% ')
    
    if val_loss < best_val_loss:
        print(f'\tval_loss({val_loss:.3f}) < best_val_loss({best_val_loss:.3f})')
        print(f'\tkeep training, best_val_loss is replaced to {val_loss:.3f}')
        best_val_loss = val_loss
        n_violence = 0
        torch.save(model.state_dict(), f'{save_dir}/my_model6.pt')
    
    else:
        n_violence+=1
        print(f'\tval_loss({val_loss:.3f}) >= best_val_loss({best_val_loss:.3f})')
        print(f'\tModel is not updated and n_violence is increased. n_violence={n_violence}')
        
        
    if n_violence >= n_patience:
        print(f'>> n_violence={n_patience}. Stop training!\n')
        break
    

Epoch: 01 | Time: 0m 3s
	Trn Loss: 0.299 | Val Loss: 0.159 | Val Acc: 95.060% 
	val_loss(0.159) < best_val_loss(inf)
	keep training, best_val_loss is replaced to 0.159
Epoch: 02 | Time: 0m 7s
	Trn Loss: 0.131 | Val Loss: 0.137 | Val Acc: 95.980% 
	val_loss(0.137) < best_val_loss(0.159)
	keep training, best_val_loss is replaced to 0.137
Epoch: 03 | Time: 0m 16s
	Trn Loss: 0.099 | Val Loss: 0.128 | Val Acc: 96.090% 
	val_loss(0.128) < best_val_loss(0.137)
	keep training, best_val_loss is replaced to 0.128
Epoch: 04 | Time: 0m 22s
	Trn Loss: 0.081 | Val Loss: 0.102 | Val Acc: 96.910% 
	val_loss(0.102) < best_val_loss(0.128)
	keep training, best_val_loss is replaced to 0.102
Epoch: 05 | Time: 0m 22s
	Trn Loss: 0.069 | Val Loss: 0.105 | Val Acc: 96.940% 
	val_loss(0.105) >= best_val_loss(0.102)
	Model is not updated and n_violence is increased. n_violence=1
Epoch: 06 | Time: 0m 21s
	Trn Loss: 0.060 | Val Loss: 0.134 | Val Acc: 96.330% 
	val_loss(0.134) >= best_val_loss(0.102)
	Model is not 